In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append(r"C:\Users\passenh\Documents\Harris\yr2_q1\ml\project\InterpretableDimRed")

In [3]:
import numpy as np 
import numpy.linalg as la
from InterpretableDimRed.utils import pca_svd_utils as psa
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
#Update the data directory to reflect your local machine
data_directory = "C:\\Users\\passenh\\Documents\\Harris\\yr2_q1\\ml\\project\\InterpretableDimRed\\InterpretableDimRed\\RawData\\"

# Building Interpretable Directions (Gu and Chipman)
Predict if patients will have heart disease or not based on 13 features.
For more info on this dataset see: https://archive.ics.uci.edu/ml/datasets/Heart+Disease

Chipman and Gu suggest taking the correlation between principle components and the original features, and applying a sort of thresholding algorithm to reset the coefficients in a way that may make them more interpretable. Among all the PC directions, for k = 1, 2, ... p, identify the k non-zero elements of the PC direction vector. Take the k elements with the largest absolute value, and set it equal to +1/-1 / sqrt(k); set all other elements equal to zero. Then find the vector with k elements which is "closest" to the original PC direction vector, and this is the "interpretable" direction.    

Their preprocessing includes removing categorical variables.

### Preprocessing

In [4]:
heart_data = pd.read_csv(data_directory + "heartDisease.csv")
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
#remove categorical data
cont_vars = ["age", "trestbps", "chol", "thalach", "oldpeak"]

heart_data = heart_data[cont_vars + ["target"]]
heart_data.head()

,age,trestbps,chol,thalach,oldpeak,target
0,63,145,233,150,2.3,1
1,37,130,250,187,3.5,1
2,41,130,204,172,1.4,1
3,56,120,236,178,0.8,1
4,57,120,354,163,0.6,1


In [6]:
#normalize
heart_norm = psa.normalize(heart_data, cont_vars)
heart_norm.head()

,age,trestbps,chol,thalach,oldpeak,target
0,0.952197,0.763956,-0.256334,0.015443,1.087338,1
1,-1.915313,-0.092738,0.072199,1.633471,2.122573,1
2,-1.474158,-0.092738,-0.816773,0.977514,0.310912,1
3,0.180175,-0.663867,-0.198357,1.239897,-0.206705,1
4,0.290464,-0.663867,2.082050,0.583939,-0.379244,1


In [7]:
#separate the X and Y matrices
X = heart_norm.iloc[:,:-1]
Y = heart_data.loc[:,"target"]

#set all the zeros to negative one
Y[Y == 0] = -1

### Get principle components, and see correlations. 

In [8]:
pca = PCA(n_components=5, svd_solver='full')
pca.fit(X.T)

components = pca.components_

In [9]:
psa.corr_pc_features(X, pca.components_)

,PC1,PC2,PC3,PC4,PC5
age,-0.668345,0.318651,-0.361172,0.377165,0.01806
trestbps,-0.246095,-0.270005,-0.626999,-0.542521,0.01806
chol,-0.048788,0.757139,0.181113,-0.460957,0.01806
thalach,0.890973,-0.116014,-0.035809,0.111126,0.01806
oldpeak,-0.602736,-0.414141,0.515593,-0.142420,0.01806


In absolute magnitude terms the highest correlated variable is "thalach", which corresponds to, with age as the next most correlated variable. 

### Run Chipman and Gu's algorithm

In [10]:
pc1 = components[0]

def c_and_g_algo(PC_vector):
    '''
    Runs the following algorithm as described by Chipman and Gu. Uses squared distance as a
    measure of closeness rather than angle closeness. 
    
    Takes the non-zero elements of the PC direction vector. 
    For K = 1 through all the non zero elements,
    Takes the k elements with the largest absolute value and sets them equal to
        plus or minus one / square root of k
    Sets all the other elements equal to zero
    Finds the vector with k elements closest to the original PC direction vector

    
    Inputs: 
        PC_vector (array) One principle component direction vector
        
    Returns: 
        closest (array) One interpretable component direction vector
        factor (float): the norm of the closest component direction vector
    '''
    closest = np.zeros((len(PC_vector)))
    factor = 1
    p = np.count_nonzero(PC_vector)
    
    signs = np.sign(PC_vector)   
    in_order = np.sort(np.abs(PC_vector.copy()))
    
    for k in range(1, p):
        magnitude = np.abs(PC_vector)
        largest = in_order[-k:]
        
        for element in largest:
            magnitude[magnitude == element] = 1
        
        magnitude[magnitude != 1] = 0
        
        interpretable = magnitude * signs 
        interpretable_factor = np.sqrt(k)
        
        distance = psa.squared_error((interpretable/interpretable_factor), PC_vector)
        prev_distance = psa.squared_error(closest, PC_vector)
        
        if distance < prev_distance:
            closest = interpretable
            factor = interpretable_factor
            
    return closest, factor

In [11]:
pcalpha, pc1factor = c_and_g_algo(pc1)
pcalpha, pc1factor

(array([-1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
        -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,
        -1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,
        -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,
        -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,
         1., -0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,
         1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
         1., -1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
         1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,  1.,  1.,
        -1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1., -1.,
         1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,
        -1., -1.,  1.,  1., -1., -1., -1.,  1.,  1.

In [12]:
interp = np.zeros(components.shape)

for i, row in enumerate(components):
    pcrow, pcrowfac = c_and_g_algo(row)
    interp[i] = pcrow / pcrowfac
    

In [13]:
psa.corr_pc_features(X, interp)

,PC1,PC2,PC3,PC4,PC5
age,-0.531033,0.209609,-0.320563,0.323531,-0.077477
trestbps,-0.222609,-0.199367,-0.490869,-0.394924,-0.004596
chol,-0.033760,0.591408,0.139121,-0.367927,-0.004176
thalach,0.738177,-0.162071,0.017907,0.096722,0.104106
oldpeak,-0.528734,-0.310454,0.380813,-0.046550,-0.070139


In [14]:
psa.corr_pc_features(X, pca.components_)

,PC1,PC2,PC3,PC4,PC5
age,-0.668345,0.318651,-0.361172,0.377165,0.01806
trestbps,-0.246095,-0.270005,-0.626999,-0.542521,0.01806
chol,-0.048788,0.757139,0.181113,-0.460957,0.01806
thalach,0.890973,-0.116014,-0.035809,0.111126,0.01806
oldpeak,-0.602736,-0.414141,0.515593,-0.142420,0.01806


In [15]:
interp.T @ interp

array([[ 0.01655629, -0.00331126, -0.00993377, ...,  0.00993377,
         0.01655629,  0.00331126],
       [-0.00331126,  0.01655629,  0.00993377, ...,  0.00331126,
        -0.00331126, -0.00331126],
       [-0.00993377,  0.00993377,  0.01655629, ..., -0.00331126,
        -0.00993377, -0.00993377],
       ...,
       [ 0.00993377,  0.00331126, -0.00331126, ...,  0.01655629,
         0.00993377, -0.00331126],
       [ 0.01655629, -0.00331126, -0.00993377, ...,  0.00993377,
         0.01655629,  0.00331126],
       [ 0.00331126, -0.00331126, -0.00993377, ..., -0.00331126,
         0.00331126,  0.01655629]])